<a href="https://colab.research.google.com/github/Wangadeveloper/artificial-intelligence/blob/main/Your_First_Chess_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

fide_google_efficiency_chess_ai_challenge_path = kagglehub.competition_download('fide-google-efficiency-chess-ai-challenge')

print('Data source import complete.')


# FIDE & Google Efficient Chess AI Challenge

Welcome! This notebook will familiarize you with using competition's environment, creating an agent, and submitting your first chess bot!

In [ ]:
# first let's make sure you have internet enabled
import requests
requests.get('http://www.google.com',timeout=10).ok

#### If you don't have internet access (it doesn't say "True" above)
1. make sure your account is Phone Verified in [account settings](https://www.kaggle.com/settings)
2. make sure internet is turned on in Settings -> Turn on internet

In [ ]:
%%capture
# ensure we are on the latest version of kaggle-environments
!pip install --upgrade kaggle-environments

In [ ]:
# Now let's set up the chess environment!
from kaggle_environments import make
env = make("chess", debug=True)

In [ ]:
# this should run a game in the environment between two random bots
# NOTE: each game starts from a randomly selected opening
result = env.run(["random", "random"])
env.render(mode="ipython", width=1000, height=1000)

### Creating your first agent
Now let's create your first agent! The environment has the [Chessnut](https://github.com/cgearhart/Chessnut) pip package installed and we'll use that to parse the board state and generate moves.

In [ ]:
%%writefile main.py
from Chessnut import Game
import random
class MCTSNode:
    def __init__(self, game, parent=None):
        self.game = game  # Chessnut Game object
        self.parent = parent
        self.children = []
        self.visits = 0
        self.wins = 0
        self.untried_moves = list(self.game.get_moves())

    def select(self):
        return max(self.children, key=lambda child: child.ucb1(self.visits))

    def expand(self):
        if not self.untried_moves:
            return None
        move = self.untried_moves.pop()
        game_copy = Game(self.game.get_fen())
        game_copy.apply_move(move)
        child_node = MCTSNode(game_copy, parent=self)
        self.children.append(child_node)
        return child_node

    def simulate(self):
        game_copy = Game(self.game.get_fen())
        while not (game_copy.status in [Game.CHECKMATE, Game.STALEMATE]):
            moves = list(game_copy.get_moves())
            if not moves:
                break
            move = random.choice(moves)
            game_copy.apply_move(move)
        return 1 if game_copy.status == Game.CHECKMATE else 0

    def backpropagate(self, result):
        self.visits += 1
        self.wins += result
        if self.parent:
            self.parent.backpropagate(result)

    def ucb1(self, total_simulations):
        if self.visits == 0:
            return float('inf')
        return (self.wins / self.visits) + 2 * ((total_simulations ** 0.5) / (1 + self.visits))

# Define the MCTS with DP logic
def mcts_with_dp(game, simulations=1000):
    # Use MCTS logic from earlier, adjusted for Chessnut's board
    # Note: Modify board interactions to align with Chessnut methods.
    root_node = MCTSNode(game)
    for _ in range(simulations):
        node = root_node
        # Selection: Traverse the tree
        while node.children:
            node = node.select()

        # Expansion: Add child nodes for unexplored moves
        if node.untried_moves:
            node = node.expand()

        # Simulation: Simulate the game
        result = node.simulate()

        # Backpropagation: Update tree with results
        node.backpropagate(result)

    # Select the best move
    best_move = max(root_node.children, key=lambda c: c.visits).move
    return best_move

def chess_bot(obs):
    """
    Enhanced chess bot using MCTS with DP for decision-making.

    Args:
        obs: An object with a 'board' attribute representing the current board state as a FEN string.

    Returns:
        A string representing the chosen move in UCI notation (e.g., "e2e4")
    """
    game = Game(obs.board)
    # Integrate MCTS with DP for move selection
    return mcts_with_dp(game, simulations=1000)


### Testing your agent

Now let's see how your agent does againt the random agent!

In [ ]:
result = env.run(["main.py", "random"])
print("Agent exit status/reward/time left: ")
# look at the generated replay.json and print out the agent info
for agent in result[-1]:
    print("\t", agent.status, "/", agent.reward, "/", agent.observation.remainingOverageTime)
print("\n")
# render the game
env.render(mode="ipython", width=1000, height=1000)

# To Submit:
1. Download (or save) main.py
2. Go to the [submissions page](https://www.kaggle.com/competitions/fide-google-efficiency-chess-ai-challenge/submissions) and click "Submit Agent"
3. Upload main.py
4. Press Submit!

Now doubt you are already thinking of ways this bot could be improved! Go ahead and fork this notebook and get started! ♟️

# Submitting Multiple files
### (or compressing your main.py)

Set up your directory structure like this:
```
kaggle_submissions/
  main.py
  <other files as desired>
```

You can run `tar -czf submission.tar.gz -C kaggle_submissions .` and upload `submission.tar.gz`